# Imports

In [12]:
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import plotly.express as px
import pandas
import numpy as np
import random
from numpy.random import default_rng
import datetime

# df_edu

In [ ]:
df_edu = pandas.read_csv('data/states_all.csv').iloc[:, 1:]
df_edu.head()

# df_ski

In [ ]:
df_ski = pandas.read_csv('data/European_Ski_Resorts.csv')  #.drop("Unnamed: 0", axis=1)

# Data Table 1

In [ ]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = Dash(__name__, )  # external_stylesheets=[dbc.themes.BOOTSTRAP]

app.layout = html.Div(
    dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df_edu.columns],
        data=df_edu.to_dict('records'),
        filter_action="native",
        sort_action="native",
        export_format="csv",
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'fontWeight': 'bold',
            'font-family': 'Arial',
            'color': 'lightgrey'
        },
        style_data={
            'font-family': 'Arial',
            'color': 'grey',
            'backgroundColor': 'rgb(50, 50, 50)',
        }
    )
)

app.run_server(debug=True)


# ASSIGNMENT: Data Table

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H2(id="title", style={"text-align": "center"}),
    html.P("Select Country Below:"),
    dcc.Dropdown(
        id="country_dropdown",
        options=df_ski['Country'].unique(),
        value="Andorra",
        placeholder="Select a country",
        className="dbc"
    ),
    html.Br(), html.Br(),
    html.P("Select Elevation:"),
    dcc.Slider(
        id="elevation_slider",
        min=0,
        max=4000,
        step=500,
        value=500,
        marks={i: {"label": f"{i} m", "style": {"fontSize": 16}}
               for i in range(0, 4001, 500)},
        className="dbc"
    ),
    html.Br(), html.Br(),
    html.Div(id="output_div")

])


@app.callback(
    Output("title", "children"),
    Output("output_div", "children"),
    Input("country_dropdown", "value"),
    Input("elevation_slider", "value"),
)
def elevation_table(country: str, elevation: int):
    if not country and not elevation:
        raise PreventUpdate

    df = df_ski.query("HighestPoint > @elevation and Country == @country")

    title = f"Ski resorts in {country} with elevation > {elevation}"

    table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        export_format="csv",
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'fontWeight': 'bold',
            'font-family': 'Arial',
            'color': 'lightgrey'
        },
        style_data={
            'font-family': 'Arial',
            'color': 'grey',
            'backgroundColor': 'rgb(50, 50, 50)',
        }
    )

    return title, table


app.run_server(debug=True)

# Conditional Callbacks

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

df_conditional = (
    pandas.read_csv("data/states_all.csv")
    .iloc[:, 1:]
    .groupby("STATE", as_index=False)
    .agg({"TOTAL_EXPENDITURE": "mean"})
    .sort_values("TOTAL_EXPENDITURE", ascending=False)
)

app.layout = html.Div([
    dcc.RadioItems(
        id="output_picker",
        options=["bar", "table"],
        value="bar",
    ),
    html.Div(id="output_div")
])


@app.callback(
    Output(component_id="output_div", component_property="children"),
    Input(component_id="output_picker", component_property="value")
)
def output_generator(output_style: str):
    if output_style == "bar":
        output = dcc.Graph(figure=px.bar(df_conditional, x="STATE", y="TOTAL_EXPENDITURE"))
    else:
        output = dash_table.DataTable(
            columns=[{"name": i, "id": i} for i in df_conditional.columns],
            data=df_conditional.to_dict('records'),
            filter_action="native",
            sort_action="native",
            export_format="csv",
            style_header={
                'backgroundColor': 'rgb(30, 30, 30)',
                'fontWeight': 'bold',
                'font-family': 'Arial',
                'color': 'lightgrey'
            },
            style_data={
                'font-family': 'Arial',
                'color': 'grey',
                'backgroundColor': 'rgb(50, 50, 50)',
            }
        )

    return output


app.run_server(debug=True)


# Chained Callbacks

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

df_edu_chain = (
    df_edu.assign(
        Avg_Math=lambda df: df[['AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE']].mean(axis=1),
        Avg_Reading=lambda df: df[['AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']].mean(axis=1),
    )
)

# TODO: Add chained callbacks here
all_options = {}

# Interactive Cross-filtering

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

df_cf = (
    df_edu
    .rename({
        "AVG_MATH_4_SCORE": "Math 4 grade",
        "AVG_MATH_8_SCORE": "Math 8 grade",
        "AVG_READING_4_SCORE": "Reading 4 grade",
        "AVG_READING_8_SCORE": "Reading 8 grade",
    }, axis=1)
    .assign(expenditure_per_student=lambda df: df['TOTAL_EXPENDITURE'] / df['GRADES_ALL_G'])
)

app.layout = html.Div([
    dbc.Row(html.H1("Education Performance & Expenditure", style={"text-align": "center"})),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dcc.Markdown("Select an X Column"),
                dcc.RadioItems(
                    id="score-radio",
                    options=[
                        {"label": "Math 4 grade", "value": "Math 4 grade"},
                        {"label": "Math 8 grade", "value": "Math 8 grade"},
                        {"label": "Reading 4 grade", "value": "Reading 4 grade"},
                        {"label": "Reading 8 grade", "value": "Reading 8 grade"},
                    ],
                    value="Math 8 grade",
                ),
                html.Hr(),
                dcc.Markdown("Select a Y Column"),
                dcc.RadioItems(
                    id="score-radio2",
                    options=[
                        {"label": "Math 4 grade", "value": "Math 4 grade"},
                        {"label": "Math 8 grade", "value": "Math 8 grade"},
                        {"label": "Reading 4 grade", "value": "Reading 4 grade"},
                        {"label": "Reading 8 grade", "value": "Reading 8 grade"},
                    ],
                    value="Math 8 grade",
                )
            ])
        ], width=2),
        dbc.Col(
            dcc.Graph(id="cross-filter-scatter", hoverData={"points": [{"customdata": "CALIFORNIA"}]}),
        ),
        dbc.Col(dcc.Graph(id="x-line"))
    ]),
    dbc.Row(html.P("Third row of app Layout")),
])

@app.callback(
    Output("cross-filter-scatter", "figure"),
    Input("score-radio", "value"),
    Input("score-radio2", "value"),
)
def score_scatter(x, y):
    fig = px.scatter(
        df_cf.query("YEAR == 2013"),
        x=x,
        y=y,
        hover_name="STATE",
        custom_data=["STATE"],
    )
    
    return fig 

@app.callback(
    Output("x-line", "figure"),
    Input("cross-filter-scatter", "hoverData")
)
def update_line(hover_data: dict):
    state_name = hover_data['points'][0]['customdata'][0]
    df = df_cf.query("STATE == @state_name")
    
    fig = px.line(
        df,
        x="YEAR",
        y="expenditure_per_student",
        title=f"Expenditure per student in {state_name.title()}",
    ).update_xaxes(showgrid=False)
    
    return fig
    
if __name__ == "__main__":
    app.run_server(debug=True)

# Real-Time Updates: Stand Deviation 

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])
rng = default_rng(42)

app.layout = html.Div([
    dbc.Row(html.H1("Normal Distribution Simulator", style={"text-align": "center"})),
    dbc.Row(dbc.Col(dcc.Graph(id="random-data-scatter"))),
    dcc.Interval(id='refresh-data-interval', interval=1000, n_intervals=0),
])

@app.callback(
    Output(component_id="random-data-scatter", component_property="figure"),
    Input(component_id="refresh-data-interval", component_property="n_intervals")
)
def rand_hist(n_intervals):
    mean, stddev = 100, 10
    fig = px.histogram(
        x=rng.normal(mean, stddev, size=1000), 
        title=f"Random Histogram has run {n_intervals} times")
    
    return fig

if __name__ == "__main__":
    app.run_server(debug=True, port=8052)

# Real-Time Updates: Live Mock Data

In [14]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

# Create an empty DataFrame with columns
columns = ["customer_id", "food_order", "sale_amount", "t"]
core_df = pandas.DataFrame(columns=columns)

def append_row() -> pandas.DataFrame:
    """
    adds a random row to the core_df
    
    :return: 
    """
    customer_id = random.randint(1, 1000) 
    food_order = random.choice(["salad", "sushi", "pizza", "taco"])
    sale_amount = round(random.uniform(5.0, 50.0), 2)
    t = datetime.datetime.now() 
    row = [customer_id, food_order, sale_amount, t]
    print(row)
    temp = pandas.DataFrame([row], columns=columns)
    merge = pandas.concat([core_df, temp])
    return merge
    

app.layout = html.Div([
    dbc.Row(html.H1("Real-Time Sales Tracker", style={"text-align": "center"})),
    dbc.Row(dbc.Col(dcc.Graph(id="sales-price-line"))),
    dcc.Interval(id='refresh-data-interval', interval=1000, n_intervals=0)
])


@app.callback(
    Output(component_id="sales-price-line", component_property="figure"),
    Input(component_id="refresh-data-interval", component_property="n_intervals")
)
def sales_line(n_intervals):
    global core_df
    core_df = append_row()
    fig = px.line(
        (
            core_df
            .assign(t=pandas.to_datetime(core_df['t'], unit='s'))
            .rename({"sale_amount": "Sale"}, axis=1)
         ),
        x="t",
        y="Sale",
        title=f"Sales as of {datetime.datetime.now()}",
    )
    return fig 

if __name__ == "__main__":
    app.run_server(debug=True, port=8052)